# Immagini di oggetti

Le immagini degli oggetti usate qui sono fornite dal seguente articolo:

Cooper, P.S., Colton, E., Bode, S. et al. Standardised images of novel objects created with generative adversarial networks. Scientific Data 10, 575 (2023). https://doi.org/10.1038/s41597-023-02483-7

All images were standardized in terms of contrast, luminance, …

Selected the data set of familiar objects (400 images). Removed the background.


## Obiettivo

Lo scopo di questo script è quello di creare un "collage" usando l'immagine di sfondo specificata (`background_orange.png` o `background_white.png`) e ciascuna delle immagini contenute nel folder `objects_folder`. In output vengono prodotte tante immagine quante sono le immainini contenute nella cartella `objects_folder`.

Le immagini contenute in `objects_folder` devono avere uno sfondo trasparente. La grandezza di tali immagini è irrilevante in quanto lo scipt procede ad uno rescaling.

TODO Trovare un modo per far sì che la coordinata y delle mani nell'immagine di backgroud sia compatibile con la posizione degli oggetti determinata dal presente script.

**Istruzioni ai soggetti** La posizione delle mani, nei termini della coordinata y, deve essere pari a 1/4 dell'altezza della foto, partendo dal basso.

---

## Design

Ogni sessione ha 50 prove. In ciascuna prova ci sono 2 stimoli: organge e white. Quindi, in totale, ci servono 100 immagini di oggetti per ciascuna sessione.

Se vogliamo oggetti diversi per "surprise" e "no surprise", abbiamo bisogno di 200 immagini di oggetti "old".  

Nel compito di memoria useremo lo stesso numero di oggetti "new". Quindi, in totale ci servono 400 immagini di oggetti.

In conclusione, in ciascuna prova della condizione "surprise" e "no surprise" usiamo immagini di oggetti diversi.

Invece, usiamo le stesse immagini degli oggetti per le condizioni "self" e "stranger".

## Image creation

Rimozione dello sfondo.

```bash
for file in *.jpg; do
  convert "$file" -fuzz 10% -transparent white "${file%.jpg}.png"
done
```

Per contare il numero di file in una cartella:

```bash
ls -l | grep ^- | wc -l
```

Per rimuovere i file jpeg:

```bash
rm *.jpg
```

Rinominiamo i files con la struttura img_001.png, img_002.png, ...:

```bash
a=1
for file in *.png; do
  new_name=$(printf "img_%03d.png" "$a")
  mv "$file" "$new_name"
  a=$((a + 1))
done
```

Delle 400 immagini così create, le immagini 1-200 andranno usate nel compito PRL. Sono le immagini "old". Le immagini 201-400 andranno usate nel compito di memoria come immagini "new".

Delle immagini "old" con indice 1-200, le immagini 1-100 saranno "white" e le immagini 101-200 saranno "orange".

In [27]:
from PIL import Image
import os
import random


# Definiamo la funzione che permette di fare il collage delle immagini degli oggetti sul background
def create_collages_based_on_background(
    background_path, objects_folder, output_folder, image_range, prefix
):
    # Verifica che il percorso del background sia valido
    if not os.path.exists(background_path):
        raise FileNotFoundError(
            f"Background file '{background_path}' non trovato. Verifica il percorso."
        )

    # Verifica che la cartella objects_folder esista
    if not os.path.exists(objects_folder):
        raise FileNotFoundError(
            f"La cartella '{objects_folder}' non esiste. Verifica il percorso."
        )

    # Carica l'immagine di background
    try:
        background = Image.open(background_path)
    except Exception as e:
        raise ValueError(f"Errore durante il caricamento del background: {e}")

    background_width, background_height = background.size

    # Dimensione ridotta degli oggetti (circa 2/5 della larghezza del background)
    object_target_width = background_width // 2.5

    # Filtra le immagini da objects_folder in base all'intervallo specificato
    object_images = [img for img in os.listdir(objects_folder) if img.endswith(".png")]
    selected_images = object_images[image_range[0] - 1 : image_range[1]]

    if len(selected_images) < 1:
        raise ValueError(
            f"Non ci sono immagini sufficienti nell'intervallo {image_range}."
        )

    # Crea la cartella di output specifica se non esiste
    specific_output_folder = os.path.join(output_folder, prefix)
    if not os.path.exists(specific_output_folder):
        os.makedirs(specific_output_folder)

    log_file_path = os.path.join(specific_output_folder, "collage_log.txt")
    with open(log_file_path, "w") as log_file:

        for idx, img in enumerate(selected_images, 1):
            try:
                with Image.open(os.path.join(objects_folder, img)) as obj:
                    # Ridimensiona l'immagine dell'oggetto mantenendo le proporzioni
                    obj_width, obj_height = obj.size
                    scaling_factor = object_target_width / obj_width
                    new_obj_width = int(obj_width * scaling_factor)
                    new_obj_height = int(obj_height * scaling_factor)

                    # Aggiungi la limitazione: l'altezza non può essere più di 1.5 volte la larghezza
                    if new_obj_height > 1.25 * new_obj_width:
                        new_obj_height = int(1.25 * new_obj_width)  # Limita l'altezza
                        scaling_factor = (
                            new_obj_height / obj_height
                        )  # Aggiorna il fattore di scala
                        new_obj_width = int(
                            obj_width * scaling_factor
                        )  # Ricalcola la larghezza

                    new_obj_size = (new_obj_width, new_obj_height)
                    obj = obj.resize(new_obj_size)

                    temp_background = background.copy()

                    # Determina casualmente se l'oggetto deve essere a sinistra o a destra del centro
                    x_position = random.choice([0, background_width - new_obj_size[0]])

                    # Evita che l'oggetto venga disegnato fuori dalla finestra grafica orizzontalmente
                    if x_position + new_obj_size[0] > background_width:
                        x_position = background_width - new_obj_size[0]

                    # L'altezza y è circa 1/4 dell'altezza totale dal basso
                    y_position = (
                        background_height - int(background_height / 4) - new_obj_size[1]
                    )

                    # Evita che l'oggetto venga disegnato fuori dalla finestra grafica verticalmente
                    if y_position + new_obj_size[1] > background_height:
                        y_position = background_height - new_obj_size[1]

                    # Incolla l'oggetto sul background con trasparenza
                    temp_background.paste(obj, (x_position, y_position), obj)

                    # Aggiusta il nome del file e la numerazione in base all'indice
                    collage_file_name = os.path.join(
                        specific_output_folder, f"{prefix}_img_{idx:03d}.png"
                    )
                    temp_background.save(collage_file_name)

                    # Logga le informazioni dell'immagine generata
                    log_file.write(f"{collage_file_name}: {img}\n")

            except Exception as e:
                print(f"Errore durante la gestione del file {img}: {e}")
                continue


In [28]:
# Parametri per generare le immagini
objects_folder = "objects_folder"  # Cartella con le immagini degli oggetti
output_folder = "output_folder"  # Cartella di output

# Genera 100 immagini per ciascuna combinazione di sfondo e cartella
create_collages_based_on_background(
    "background_orange.png", objects_folder, output_folder, (1, 100), "old_orange"
)
create_collages_based_on_background(
    "background_white.png", objects_folder, output_folder, (101, 200), "old_white"
)
create_collages_based_on_background(
    "background_orange.png", objects_folder, output_folder, (201, 300), "new_orange"
)
create_collages_based_on_background(
    "background_white.png", objects_folder, output_folder, (301, 400), "new_white"
)